In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
formular1_key = dbutils.secrets.get(scope='fomular1-scope', key='formuladl1-key-account')


In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", formular1_key)

In [0]:
from pyspark.sql.functions import col, lit, to_timestamp, concat, current_timestamp, to_date, substring, regexp_extract, to_timestamp
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType, TimestampType



In [0]:
lap_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                 StructField("driverId", IntegerType(), True),
                                 StructField("stop", StringType(), True),
                                 StructField("lap", IntegerType(), True),
                                 StructField("time", StringType(), True),
                                 StructField("duration", StringType(), True),
                                 StructField("milliseconds", IntegerType(), True)
                                ])

In [0]:
lap_df = spark.read.schema(lap_schema).csv(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/lap_times")

In [0]:
lap_df.show()

In [0]:
lap_df.count()

In [0]:
final_df = lap_df.withColumnRenamed("raceId", "race_id") \
                 .withColumnRenamed("driverId", "driver_id")\
                .withColumn("ingestion_date", current_timestamp())


In [0]:
final_df.show()

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")